<a href="https://colab.research.google.com/github/boothmanrylan/MSSForestDisturbances/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and installs

In [ ]:
# standard python imports
import os

In [ ]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# pip installable modules

try:
    import geemap.foliumap as geemap
except:
    !pip install geemap
    import geemap.foliumap as geemap

In [ ]:
# install and import my packages from github
from google.colab import drive, auth

drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/colaboratory

# auth.authenticate_user()

# install msslib
! if [ ! -d ./msslib ]; then git clone https://github.com/boothmanrylan/msslib.git; fi
%cd msslib
! git pull
! pip install .
%cd ../

from msslib import msslib

# install sequencer
! if [ ! -d ./sequencer ]; then git clone https://github.com/boothmanrylan/sequencer.git; fi
%cd sequencer
! git pull
! pip install .
%cd ../

import sequencer

# install MSSForestDisturbances
! if [ ! -d ./MSSForestDisturbances ]; then git clone https://github.com/boothmanrylan/MSSForestDisturbances.git; fi
%cd MSSForestDisturbances
! git pull
! pip install .
%cd ../

import mfd.postprocess
import mfd.preprocess
import mfd.utils
import mfd.get_collection
import mfd.export

# Reload

Run these blocks if the modules have been updated since the runtime was started

In [ ]:
from importlib import reload

In [ ]:
# reload msslib
%cd ./msslib
! git pull
msslib = reload(msslib)
%cd ../

In [ ]:
# reload sequencer
%cd ./sequencer/
! git pull
sequencer = reload(sequencer)
%cd ../

In [ ]:
# reload MSSForestDisturbances
%cd ./MSSForestDisturbances/
! git pull
mfd.postprocess = reload(mfd.postprocess)
mfd.preprocess = reload(mfd.preprocess)
mfd.utils = reload(mfd.utils)
mfd.get_collection = reload(mfd.get_collection)
mfd.export = reload(mfd.export)
%cd ../

# Test

In [ ]:
def get_valid_burns(events):
    years = events.distinct("year").aggregate_array("year")

    def _valid(year):
        curr_events = events.filter(ee.Filter.eq("year", year))
        exclusively_burnt = curr_events.reduce(ee.Reducer.allNonZero())
        burnt_often = curr_events.reduce(ee.Reducer.sum()).gt(1)
        return exclusively_burnt.Or(burnt_often).selfMask()

    result = ee.ImageCollection.fromImages(years.map(_valid))
    return result.reduce(ee.Reducer.anyNonZero())

def postprocess(event):
    event = mfd.postprocess.postprocess(event)
    event = mfd.postprocess.squash_extra_classes(event)
    event = mfd.postprocess.set_true_date(event)
    return event

In [ ]:
grid_split_qc = ee.FeatureCollection('users/boothmanrylan/gridSplitForestedQC')

grid_cell = grid_split_qc.filter(ee.Filter.eq("id", 48)).first().geometry().centroid(100)


In [ ]:
quebec_fires = ee.ImageCollection("users/boothmanrylan/MSSForestDisturbances/test2")

events = mfd.postprocess.filter_and_clip(quebec_fires, grid_cell)

valid_burns = get_valid_burns(events.map(postprocess))

In [ ]:
print(events.size().getInfo())

In [ ]:
Map = geemap.Map()
Map.addLayer(grid_split_qc, {}, 'Grid Split QC')
Map.addLayer(grid_cell, {}, 'Grid Cell of Interest')
Map.addLayer(events, mfd.utils.CLASS_VIS, 'Events')
Map.addLayer(valid_burns, {'min': 0, 'max': 1}, 'Valid Burns')

Map